<a href="https://colab.research.google.com/github/Valentinafoschi/text2relax/blob/main/rumori_bianchi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

questo progetto ha come obiettivo quello di generare un audio + video rilassanti a partire da una descrizione testuale.

testo --> paesaggi sonori + visivi

In [5]:
# INSTALLAZIONE LIBRERIE NECESSARIE

!pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip -q install open-clip-torch==2.24.0 moviepy==1.0.3 librosa==0.10.2.post1 soundfile==0.12.1 scipy==1.11.4 numpy==1.26.4 scikit-learn==1.4.2


di seguito importiamo tutte le librerie necessarie per il progetto


In [6]:
import os, json, random, math, colorsys, re
from typing import Dict, Any, List, Tuple

import numpy as np
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import librosa, soundfile as sf
from scipy.signal import butter, lfilter, fftconvolve
from moviepy.editor import AudioFileClip, VideoClip

import open_clip

# CREAZIONE CARTELLE BASE PROGETTO
for d in ["data","results","checkpoints"]:
    os.makedirs(d, exist_ok=True)

# seed + device
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

scegliamo le caratteristiche visive che il nostro modello deve generare e i tipi di audio che dovrà poi prevedere.
pattern scelti:
- **electric** --> lampi o scariche che attraversano lo schermo
***snow** ---> fiocchi di neve che cadono
***waves** --> onde sinusoidali lente
***ripple** --> onde circolari concentriche tipo goccia in acqua
***rotating shapes** --> cerchi, quadrati o triangoli che ruotano lentamente



In [7]:
NOISE_TYPES = ["white", "pink", "brown", "blue"]
PATTERNS = ["electric", "snow", "waves", "ripple", "rotatingShapes"]

RANGES = {
    # AUDIO
    "duration_sec": (30, 90),
    "lp_cutoff": (300, 8000),   # filtro low-pass
    "hp_cutoff": (20, 2000),    # filtro high-pass
    "reverb_mix": (0.0, 0.6),   # quantità di riverbero
    "fade_in_sec": (0.0, 5.0),  # durata del fade-in
    "fade_out_sec": (0.0, 5.0), # durata del fade-out
    "rain_level": (0.0, 1.0),   # intensità del pioggia
    "waves_level": (0.0, 1.0),  # intensità delle onde
    "ripple_level": (0.0, 1.0), # intensità delle goccie

    # VIDEO
    "hue_main": (0.0, 1.0),     # tonalità colore HSV
    "sat": (0.2, 1.0),          # saturazione colore HSV
    "value": (0.2, 1.0),        # luminosità colore HSV
    "motion_speed": (0.05, 0.6),    # velocità movimento
    "motion_amplitude": (0.1, 1.0), # ampiezza movimento
    "scene_brightness": (0.2, 1.0), # luminosità globale
    "sync_to_audio": (0.0, 1.0),    # sincronizzazione con audio

}


# funzione per normalizzare un parametro
def norm_param (name, x):
  lo, hi = RANGES[name]
  return (x - lo) / (hi - lo + 1e-9)


# funzione per denormalizzare un parametro
def denorm_param (name, x01):
  lo, hi = RANGES[name]
  return float(lo + x01 * (hi - lo))

nella cella seguente ci occupiamo del renderer audio, cioè la parte che a partire dai parametri audio previsti dal modello genera il file .wav vero e proprio.

In [10]:
SR = 22050 # quante vote viene campionato il segnale audio

# filtraggio frequenza
def _butter_filter (x, cutoff, btype):
  if cutoff <= 0:
    return x
  nyq = 0.5 * SR
  cutoff = min(cutoff, nyq-100)
  b, a = butter(4, cutoff / nyq, btype=btype)
  return lfilter(b, a, x)

# generazione rumore base
def _gen_noise (kind, n):
  if kind == "white": #frequenze uguali
    x = np.random.randn(n)
  elif kind=="pink": # più energia sulle basse frequenze
    x = _butter_filter(np.random.randn(n), 500, 'low')
  elif kind=="brown": # ancora più basse frequenze
    x = np.cumsum(np.random.randn(n)); x /= (np.abs(x).max()+1e-9)
  elif kind=="blue": # frequenze più alte
    x = _butter_filter(np.random.randn(n), 3000, 'high')
  return x.astype(np.float32)

# genera un'onda alla frequenza
def _sine(freq, n): t = np.arange(n)/SR; return np.sin(2*np.pi*freq*t).astype(np.float32)

#simula le gocce di pioggia
def _rain(n, level):
    if level<=1e-3: return np.zeros(n, np.float32)
    y = np.zeros(n, np.float32); drops = int(level*3*n/SR)
    for _ in range(drops):
        i = np.random.randint(0, n-200)
        y[i:i+200] += (np.hanning(200)*np.random.uniform(0.3,1.0)).astype(np.float32)
    return _butter_filter(y, 8000, 'low')

# genera le onde del mare
def _waves(n, level):
    if level<=1e-3: return np.zeros(n, np.float32)
    base = _sine(np.random.uniform(0.1,0.25), n)*0.3
    mod  = _sine(np.random.uniform(0.01,0.05), n)*0.7 + 0.7
    return (base*mod*level).astype(np.float32)



# FUNZIONE PRINCIPALE DI RENDERING AUDIO

def render_audio(params: Dict[str, Any], out_wav: str):
    n = int(params["duration_sec"]*SR)
    x = _gen_noise(params["noise_type"], n)
    if params["lp_cutoff"]>0: x = _butter_filter(x, params["lp_cutoff"], 'low')
    if params["hp_cutoff"]>0: x = _butter_filter(x, params["hp_cutoff"], 'high')
    x = 0.7*x + 0.2*_rain(n, params["rain_level"]) + 0.3*_waves(n, params["waves_level"])
    if params["reverb_mix"]>0:
        ir = np.exp(-np.linspace(0,1.0,int(0.12*SR))).astype(np.float32)
        wet = fftconvolve(x, ir)[:n]
        x = (1-params["reverb_mix"])*x + params["reverb_mix"]*wet
    fi = int(params["fade_in_sec"]*SR); fo = int(params["fade_out_sec"]*SR)
    if fi>0: x[:fi] *= np.linspace(0,1,fi)
    if fo>0: x[-fo:] *= np.linspace(1,0,fo)
    x /= (np.abs(x).max()+1e-9)
    sf.write(out_wav, x, SR); return out_wav